# ジェネレータで throw による状態遷移を起こすのは避ける

In [1]:
class MyError(Exception):
  pass

def my_generator():
  yield 1
  yield 2
  yield 3

it = my_generator()
print(next(it))
print(next(it))
print(it.throw(MyError('test error')))

1
2


MyError: test error

In [3]:
def my_generator():
  yield 1

  try:
    yield 2
  except MyError:
    print('Got MyError')
  else:
    yield 3

  yield 4

it = my_generator()
print(next(it))
print(next(it))
print('Before throw Error')
print(it.throw(MyError('test error')))

1
2
Before throw Error
Got MyError
4


In [4]:
class Reset(Exception):
  pass

def timer(period):
  current = period
  while current:
    current -= 1
    try:
      yield current
    except Reset:
      current = period

In [21]:
import random

def check_for_reset():
  # 外部イベントをポーリングして待つ
  if random.random() < 0.9:
    return False
  return True

def announce(remaining):
  print(f'{remaining} ticks remaining')

def run():
  it = timer(4)
  while True:
    try:
      if check_for_reset():
        current = it.throw(Reset())
      else:
        current = next(it)
    except StopIteration:
      break
    else:
      announce(current)

run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining


コードは動作するが読みずらい

In [ ]:
# イテラブルなコンテナオブジェクトを使った状態を持つクロージャで実装
# 読みやすい
class Timer:
  def __init__(self, period):
    self.current = period
    self.period = period

  def reset(self):
    self.current = self.period

  def __iter__(self):
    while self.current:
      self.current -= 1
      yield self.current

In [28]:
def run():
  timer = Timer(4)
  for current in timer:
    if check_for_reset():
      timer.reset()
    announce(current)

run()

3 ticks remaining
3 ticks remaining
2 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining
